## set path and parameters

Here we set all of the parameters that control how the notebook will behave. Ideally, the following cell would be the only thing you have to change when analyzing different datasets. Indeed, we put all parameters here under a single cell to facilitate batch processing. Most of the parameters will not make sense until we reach and discuss the corresponding steps in the pipeline. For this reason, here we discuss only some initial parameters that have a broader impact, and leave the discussion of specific parameters for later.

* `minian_path` should be the path that contains a folder named **"minian"** , under which the actual code of **minian** (.py files) reside. The default value **\.** means "current folder", which should work in most cases, unless you want to try out another version of minian that is not in the same folder as this notebook.

* `dpath` is the folder that contains the actual videos to be processed, which are usually named **"msCam\*.avi"** where \* is a number.

* `interactive` controls whether interactive plots will be shown.  Note that interactive plotting requires computation to be carried out, and thus could be very inefficient when the data are not in the memory (in particular, those steps where video is played).  However, if your video is large, you may want to set `interactive = True` for the purposes of paramater exploration, but have `in_memory` set to false.   
<div class="alert alert-info">
In practice, when you want to visualize interactive plots while figuring out the best parameters for your data, you would want in_memory=True and interactive=True, as long as your data can fit in the memory. On the other hand, once you finalize your parameters and are ready for batch processing, you want both of them to be set as False.
</div>

* `output_size` controls the relative size of all the plots on a scale of 0-100 percent, though it can be set to values >100 without any problem. Adjust this to please your eye. 

* `param_save_minian` specifies how data is to be saved. `dpath` is the folder path defining where you want the data to be saved. We recommand using the same dpath as where you load the imaging data from. `fname` is the name of your dataset. In `backend`, `'zarr'` is designed for parallel and out-of-core computation, and is the current default for minian. That said, its support is experimental for now and does not support incremental writing (i.e. it will be a pain to update part of an exsisting dataset). `meta_dict` is a `dictionary` that is used to construct meta data for the final labeled data structure.

<div class="alert alert-info">
The defult meta_dict is assumes data is stored in heirarchiically arranged folders, as shown below. We <strong> recommand </strong> users to structure their data like this so that they don't have to adjust this meta_dict setting.  This is also the default manner in which Miniscope data is saved. However, if you already have a preferred way to store your data, you can simply change the value of meta_dict in this parameter to suit your needs!
</div>

**recommended folder structure**

![Folder Structure](img/folder_structure.png)

The default value can be read as follows: the name of the last folder (`values`=-1) in `dpath` (the folder that directly contains the videos) will be used to designate the value of a field named **'session_id'**. The name of the second-to-last folder (`values`=-2) in `dpath` will be used to designate the value for **'session'** and so on. Both the `keys` (field names) and `values` (numbers indicating which level of folder name should be used) of `meta_dict` can be modified to suit your data structure. `overwrite` is a boolean value controlling whether the data is overwritten if a file already exsists. We set it to `True` here so you can easily play with the demo multiple times, but **use extreme caution** with this during actual analysis -- in addition to erasing prior data that may be important to you, under certain circumstances it is possible for existing file structures to cause compatibablity issues and data will be saved improperly.  If you want to re-analyze a video from scratch using different parameters, it is recommended that you delete existing data first.

In [ ]:
import os
import numpy as np

#Set up Initial Basic Parameters#
minian_path = "."
dpath = "./demo_movies"
subset = dict(frame=slice(0,None))
subset_mc = None

# try to retrieve interactie from the os env
interactive = os.getenv("interactive", True)

output_size = 100
param_save_minian = {
    'dpath': dpath,
    'fname': 'minian',
    'backend': 'zarr',
    'meta_dict': dict(session_id=-1, session=-2, animal=-3),
    'overwrite': True}

#Pre-processing Parameters#
param_load_videos = {
    'pattern': 'msCam[0-9]+\.avi$',
    'dtype': np.uint8,
    'downsample': dict(frame=2,height=1,width=1),
    'downsample_strategy': 'subset'}
param_denoise = {
    'method': 'median',
    'ksize': 7}
param_background_removal = {
    'method': 'tophat',
    'wnd': 15}

#Motion Correction Parameters#
subset_mc = None
param_estimate_shift = {
    'dim': 'frame',
    'max_sh': 20}

#Initialization Parameters#
param_seeds_init = {
    'wnd_size': 1000,
    'method': 'rolling',
    'stp_size': 500,
    'nchunk': 100,
    'max_wnd': 15,
    'diff_thres': 2}
param_pnr_refine = {
    'noise_freq': 0.1,
    'thres': 1,
    'med_wnd': None}
param_ks_refine = {
    'sig': 0.05}
param_seeds_merge = {
    'thres_dist': 5,
    'thres_corr': 0.7,
    'noise_freq': 0.1}
param_initialize = {
    'thres_corr': 0.8,
    'wnd': 15,
    'noise_freq': 0.1}

#CNMF Parameters#
param_get_noise = {
    'noise_range': (0.1, 0.5),
    'noise_method': 'logmexp'}
param_first_spatial = {
    'dl_wnd': 15,
    'sparse_penal': 0.1,
    'update_background': True,
    'normalize': True,
    'zero_thres': 'eps'}
param_first_temporal = {
    'noise_freq': 0.1,
    'sparse_penal': 0.05,
    'p': 1,
    'add_lag': 20,
    'use_spatial': False,
    'jac_thres': 0.2,
    'zero_thres': 1e-8,
    'max_iters': 200,
    'use_smooth': True,
    'scs_fallback': False,
    'post_scal': True}
param_first_merge = {
    'thres_corr': 0.8}
param_second_spatial = {
    'dl_wnd': 15,
    'sparse_penal': 0.005,
    'update_background': True,
    'normalize': True,
    'zero_thres': 'eps'}
param_second_temporal = {
    'noise_freq': 0.1,
    'sparse_penal': 0.05,
    'p': 1,
    'add_lag': 20,
    'use_spatial': False,
    'jac_thres': 0.2,
    'zero_thres': 1e-8,
    'max_iters': 500,
    'use_smooth': True,
    'scs_fallback': False,
    'post_scal': True}
